### Simple Markov Model – Evaluation Setup

Simulate a communication model with two states: Working (W) and Failed (F) using transition probabilities. Track how these failures affect coordination and performance.

In [7]:
from mpe2 import simple_spread_v3
from failure_api.communication_models import BaseMarkovModel
from failure_api.wrappers import CommunicationWrapper
from pettingzoo.utils import aec_to_parallel
import numpy as np
import time
import matplotlib.pyplot as plt

N = 3
episodes = 30
seeds = [42, 123]
max_cycles = 24

agent_ids = [f"agent_{i}" for i in range(N)]
# Transition matrix:
# P[0] = [stay_disconnected, become_connected]
# P[1] = [become_disconnected, stay_connected]
simple_matrix = np.array([[0.8, 0.2],  # Disconnected → (D, C)
                          [0.3, 0.7]])  # Connected → (D, C)

markov_model = BaseMarkovModel(
    agent_ids=agent_ids,
    default_matrix=simple_matrix,
)
markov_env = simple_spread_v3.env(N=N, max_cycles=max_cycles)
wrapped_markov_env = CommunicationWrapper(markov_env,
                                          failure_models=[markov_model])

parallel_markov = aec_to_parallel(wrapped_markov_env)

all_rewards = []
step_times = []
episode_broken = False

for seed in seeds:
    for episode in range(episodes):
        obs, _ = parallel_markov.reset(seed=seed)
        total_reward = 0
        start_time = time.perf_counter()
        
        for step in range(max_cycles):
            if not parallel_markov.agents:
                break
                
            try:
                actions = {
                    agent: parallel_markov.action_space(agent).sample() 
                    for agent in parallel_markov.agents
            }
                
                obs, rewards, terminations, truncations, infos = parallel_markov.step(actions)
                
                total_reward += sum(rewards.values())
            
                if all(terminations.values()) or all(truncations.values()):
                    break
                
            except KeyError as e:
                if not episode_broken:
                    print(f"⚠️ Episode {episode} (Seed {seed}) interrupted due to KeyError on {e}")
                    episode_broken = True
                break
            
        step_duration = (time.perf_counter() - start_time) / max_cycles
        step_times.append(step_duration * 1000)
        all_rewards.append(total_reward)
        
    print("=== Simple Markov Model Summary ===")
    print(f"Total episodes: {len(all_rewards)}")
    print(f"Avg Reward: {np.mean(all_rewards):.2f} ± {np.std(all_rewards):.2f}")
    print(f"Avg Step Time: {np.mean(step_times):.3f} ms")



⚠️ Episode 0 (Seed 42) interrupted due to KeyError on 'agent_0'
=== Simple Markov Model Summary ===
Total episodes: 30
Avg Reward: -66.07 ± 6.72
Avg Step Time: 1.925 ms
=== Simple Markov Model Summary ===
Total episodes: 60
Avg Reward: -77.24 ± 13.73
Avg Step Time: 1.698 ms


#### Extended Markov Model
Introduce richer state dynamics with three communication states, each with different masking effects to simulate partial degradation in communication.

